In [87]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import normalize
from sklearn.model_selection import cross_val_score

## Puntaje
Your submission scored 0.84053, which is not an improvement of your best score. Keep trying!

In [88]:
df1 = pd.read_csv('../data/features_basicas.csv', low_memory=False, index_col='person')
df2 = pd.read_csv('../data/features_checkouts.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('../data/features_vistos.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('../data/feature_dispositivo_mas_usado.csv', low_memory=False, index_col='person')
df5 = pd.read_csv('../data/feature_dia_mas_activo.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('../data/feature_compra_onsite.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('../data/feature_cantidad_de_eventos_en_intervalos.csv', low_memory=False, index_col='person')
df8 = pd.read_csv('../data/feature_dispositivos_storage.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('../data/featureUsuarioSonDeSaoPablo.csv', low_memory=False, index_col='person')
df10 = pd.read_csv('../data/featureCantidadMaximaPersonaVeUnProducto.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('../data/features_basicas_ult_quin.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('../data/features_ultima_ocurrencia_cada_evento.csv', low_memory=False, index_col='person')
df13 = pd.read_csv('../data/days_elapsed_from_last_event.csv', low_memory=False, index_col='person')
df14 = pd.read_csv('../data/featureUsuarioRealiza30EventosEn20MinsLosUltimos2Dias.csv', low_memory=False, index_col='person')
df15 = pd.read_csv('../data/features_ultimo_checkout.csv', low_memory=False, index_col='person')
df16 = pd.read_csv('../data/diferencia_de_count_de_eventos.csv', low_memory=False, index_col='person')
df17 = pd.read_csv('../data/top_10_celulares.csv', low_memory=False, index_col='person')
df18 = pd.read_csv('../data/feature_cantidad_de_eventos_en_intervalos_ult_quin.csv', low_memory=False, index_col='person')
df19 = pd.read_csv('../data/semana_3_dias_antes_features.csv', low_memory=False, index_col='person')
#df20 = pd.read_csv('../data/features_ultima_ocurrencia_cada_evento.csv', low_memory=False, index_col='person')


labels = pd.read_csv('../data/labels_training_set.csv', low_memory=False, index_col='person')

df_unidos = df1.join(df2, how='inner')
df_unidos = df_unidos.join(df3, how='inner')
df_unidos = df_unidos.join(df4, how='inner')
df_unidos = df_unidos.join(df5, how='inner')
df_unidos = df_unidos.join(df6, how='inner')
df_unidos = df_unidos.join(df7, how='inner')
df_unidos = df_unidos.join(df8, how='inner')
df_unidos = df_unidos.join(df9, how='inner')
df_unidos = df_unidos.join(df10, how='inner')
df_unidos = df_unidos.join(df11, how='inner')
df_unidos = df_unidos.join(df12, how='inner')
df_unidos = df_unidos.join(df13, how='inner')
df_unidos = df_unidos.join(df14, how='inner')
df_unidos = df_unidos.join(df15, how='inner')
# Al que tenga "diferencia_de_count_de_eventos.csv" hay que rellenarlo
df_unidos = df_unidos.join(df16).fillna(0)
df_unidos = df_unidos.join(df17, how='inner')
df_unidos = df_unidos.join(df18, how='inner')
df_unidos = df_unidos.join(df19, how='inner')
#df_unidos = df_unidos.join(df20, how='inner')


#  Le asigno las features al subset de usuarios que nos da Trocafone para entrenar.
df_test = df_unidos.join(labels, on='person', how='inner')
df_test.shape

(19414, 102)

In [89]:
df_unidos.shape

(38829, 101)

In [90]:
df11.columns

Index(['ult_quin_cant_conversions', 'ult_quin_cant_checkouts',
       'ult_quin_cant_viewed_product', 'ult_quin_cant_searched_product',
       'ult_quin_cant_visitas', 'ult_quin_cant_leads',
       'ult_quin_cant_brand_listings', 'ult_quin_cant_generic_listings',
       'ult_quin_cant_entradas_ads', 'ult_quin_cant_entradas_buscador',
       'tasa_ult_quin_cant_checkouts', 'tasa_ult_quin_cant_conversions',
       'tasa_ult_quin_cant_viewed_product',
       'tasa_ult_quin_cant_searched_product', 'tasa_ult_quin_cant_visitas',
       'tasa_ult_quin_cant_leads', 'tasa_ult_quin_cant_brand_listings',
       'tasa_ult_quin_cant_generic_listings',
       'tasa_ult_quin_cant_entradas_ads',
       'tasa_ult_quin_cant_entradas_buscador', 'ult_quin_total_sesiones',
       'ult_quin_promedio_eventos_por_sesion'],
      dtype='object')

In [91]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=30, random_state=1) 
features_reducidas = svd.fit_transform(df_unidos)

In [92]:
df_reducido = pd.DataFrame(features_reducidas)
df_reducido.shape

(38829, 30)

In [93]:
df_svdTest = df_unidos.reset_index()

In [94]:
df_reducido["person"] = df_svdTest["person"]

In [95]:
df_reducido.set_index("person",inplace=True)

In [96]:
df_test = df_reducido.join(labels, on='person', how='inner')

In [97]:
features = df_test.columns.tolist()
features.remove('label')

In [98]:
X = df_test[features]
Y = df_test['label']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=311)

In [99]:
param_grid = { 
    'n_estimators': [x for x in range(100, 1000, 100)],
    'gamma' : [1,2,3,4,5],
}

In [100]:
xgb = XGBClassifier(random_state=80)
CV_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='roc_auc', n_jobs=4,)
CV_xgb.fit(X_train, y_train)

KeyboardInterrupt: 

In [7]:
CV_xgb.best_params_

{'gamma': 3, 'n_estimators': 100}

In [8]:
CV_xgb.best_score_
#0.8751560441726721 old score!

0.8751560441726721

In [80]:
model_xgb = XGBClassifier(scale_pos_weight=1, n_estimators=100, random_state=80, max_depth=3)
model_xgb.fit(X_train, y_train)
y_pred = model_xgb.predict_proba(X_test)
y_pred_proba = [p[1] for p in y_pred]
print(roc_auc_score(y_test, y_pred_proba))

#0.8577688383045525 old score!

0.8302642595499738


In [81]:
#  Acá quedan ordenadas las features por importancia para el modelo, aparentemente la marca más vista y la marca
#  que más compró cada usuario no es tan importante.
feature_importances = pd.DataFrame(CV_xgb.best_estimator_.feature_importances_,\
                                   index = X.columns,\
                                    columns=['importance']).sort_values('importance',ascending=False)

feature_importances.head(40)

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [82]:
df_submit = pd.read_csv('../data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_reducido, how='inner')

In [83]:
df_events.shape

(19415, 30)

In [84]:
# NOTAR QUE USE EL MODELO SIN KFOLD!, igual manera no explica que este 0.03 mas bajo del mejor!.
kaggle_pred = model_xgb.predict_proba(df_events)
proba_de_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_de_comprar)
df_submit['label'] = series.values

In [86]:
df_submit.to_csv('../submit_SVD_30Componentes.csv')